In [4]:

import pandas as pd
from textblob import TextBlob
from tqdm import tqdm_notebook as tqdm


df = pd.read_csv('GlassdoorSentimentTop5.csv', index_col = 0)


In [6]:
def add_sentiment_col2(df, col_name, on_missing_neutral = True):
    texts = df[col_name]
    pbar = tqdm(total=len(texts))
    polarity = []
    sentiment = []
    for x in texts:
        try:
            analysis = TextBlob(x).sentiment[0]
        except TypeError:
            if on_missing_neutral:
                polarity.append(0)
                sentiment.append('Neutral')
            else:
                polarity.append(None)
                sentiment.append(None)
            continue       
        polarity.append(analysis)
        if analysis > 0:
            sentiment.append('Positive')
        elif analysis == 0:
            sentiment.append('Neutral')
        else:
            sentiment.append('Negative')
        pbar.update(1)
    df[col_name+'polarity'] = polarity
    df[col_name+'sentiment'] = sentiment
    return df
df.head()

,Rating,Comments
Company,,
JLL,5,Great Opportunities. Very focused on training ...
JLL,5,Great community of individuals to work with. ...
JLL,3,Working on a client site that as some great pe...
JLL,2,The company has lots of resources and is willi...
JLL,4,"Good place to work. Great Challenges, decent p..."


In [7]:

from nltk.corpus import stopwords
from nltk.tokenize import TweetTokenizer 
import matplotlib.pyplot as plt
from collections import Counter
import string
import nltk
from tqdm import tqdm_notebook as tqdm
%matplotlib inline

def process(text, stopwords=[]):   
    text = text.lower()
    tokens = TweetTokenizer().tokenize(text)
    return [tok for tok in tokens if tok not in stopwords and not tok.isdigit()]

def get_frequent(df, col_name):
    #punct = list(string.punctuation)
    #stopword_list = stopwords.words('english') + punct + ['rt', 'via', '..', '...', '…', '‘', '’', '“', '”', '😬', '▪', '😂'] 
    tf = Counter()
    texts = df[col_name]
    pbar = tqdm(total=len(df))
    for text in texts:
        try:
            tokens = process(text)
        except:
            pass
        else:
        #tokens = process(text, stopwords = stopword_list)
            tf.update(tokens)
        pbar.update(1)
    return tf

def filter_tags(tf, tags = ['JJ', 'JJR', 'JJS']):
    pbar = tqdm(total=len(tf.most_common(1000)))
    punct = list(string.punctuation)
    stopword_list = stopwords.words('english') + punct + ['rt', 'via', '..', '...', '…', '‘', '’', '“', '”', '😬', '▪', '😂',
                                                          'https']
    for tag, count in tf.most_common(1000):
        pos = nltk.pos_tag([tag])
        if (pos[0][1] in tags) & (tag not in stopword_list) & (len(tag) > 1):
            print("{}: {} : {}".format(tag, count, pos))
        pbar.update(1)
    return

In [8]:
tf = get_frequent(df, 'summary')
filter_tags(tf, tags = ['NN', 'NNS', 'NNP', 'NNPS'])

KeyError: 'summary'